In [9]:
import os
import glob
import math

import ROOT
from hepdata_lib import Submission, Table, Variable, Uncertainty, RootFileReader

ROOT_FILENAME = "input/ZNuNuGPrePostFitPostFitEBEE.root"
OUTPUT_DIR = "hepdata_output"
ABSTRACT_FILE = None  # e.g. "abstract.txt"

CM_ENERGY_GEV = 13000.0
PAPER_TITLE = (
    "Measurement of the $Z\\gamma$ production cross section and search for "
    "anomalous neutral triple gauge couplings in pp collisions at $\\sqrt{s}=13$ TeV"
)
CMS_ANALYSIS_ID = "CMS SMP-22-009"

OBS_NAME = r"$p_{T}^{\gamma}$"
OBS_UNITS = "GeV"

USE_GARWOOD = True
POISSON_CL = 0.6827

ROUND_ROOT_TABLES = True
UNC_SIGDIGITS = 2  # CMS-style: round uncertainties to N sig digits, values to match


REGIONS = [
    {
        "name": "Figure4_EB_pTgamma",
        "location": "Figure 4 (left)",
        "root_dir": "postfit/EB_SR",
        "data_hist": "data_obs",
        "description": (
            "Post-fit reconstruction-level photon transverse momentum $p_{T}^{\\gamma}$ "
            "distribution in the ECAL barrel (EB) signal region for the full Run-2 dataset "
            "(138 fb$^{-1}$ at $\\sqrt{s}=13$ TeV)."
        ),
        "primary_mc": [
            (r"Fiducial $Z\gamma$", "mergedFiducialZNuNuG"),
            (r"$W+\gamma$", "mergedWLNuG"),
            ("Spikes", "ECAL_spikes"),
            (r"$e\to\gamma$", "eleFakes"),
        ],
        "other_mc": [
            ("Jet fakes", "jetFakes"),
            (r"Out-of-acceptance $Z(\nu\nu)\gamma$", ["ooaFixed", "OOAfixed"]),
            ("Minor backgrounds", "minor_bkg"),
        ],
        "extra_mc": [
            ("Total background", "TotalBkg"),
            ("Predicted", "TotalProcs"),
        ],
    },
    {
        "name": "Figure4_EE_pTgamma",
        "location": "Figure 4 (right)",
        "root_dir": "postfit/EE_SR",
        "data_hist": "data_obs",
        "description": (
            "Post-fit reconstruction-level photon transverse momentum $p_{T}^{\\gamma}$ "
            "distribution in the ECAL endcap (EE) signal region for the full Run-2 dataset "
            "(138 fb$^{-1}$ at $\\sqrt{s}=13$ TeV)."
        ),
        "primary_mc": [
            (r"Fiducial $Z\gamma$", "mergedFiducialZNuNuG"),
            (r"$W+\gamma$", "mergedWLNuG"),
            (r"$e\to\gamma$", "eleFakes"),
            ("Beam halo", "beamHalo"),
        ],
        "other_mc": [
            ("Jet fakes", "jetFakes"),
            (r"Out-of-acceptance $Z(\nu\nu)\gamma$", ["OOAfixed", "ooaFixed"]),
            ("Minor backgrounds", "minor_bkg"),
        ],
        "extra_mc": [
            ("Total background", "TotalBkg"),
            ("Predicted", "TotalProcs"),
        ],
    },
]


def _round_sig(x: float, sig: int) -> float:
    if x == 0 or not math.isfinite(x):
        return x
    p = sig - 1 - int(math.floor(math.log10(abs(x))))
    return round(x, p)


def _decimals_from_unc(unc: float, sig: int) -> int:
    if unc <= 0 or not math.isfinite(unc):
        return 6
    u = _round_sig(unc, sig)
    if u == 0:
        return 6
    return sig - 1 - int(math.floor(math.log10(abs(u))))


def _cms_round_value_and_unc(y, dy, sig=UNC_SIGDIGITS):
    if dy is None:
        return y, dy
    if isinstance(dy, tuple):
        dn, up = float(abs(dy[0])), float(abs(dy[1]))
        dec = _decimals_from_unc(max(dn, up), sig)
        return round(float(y), dec), (-round(dn, dec), +round(up, dec))
    dec = _decimals_from_unc(float(abs(dy)), sig)
    return round(float(y), dec), round(float(dy), dec)


def poisson_garwood_intervals(values, cl=POISSON_CL):
    alpha = 1.0 - cl
    err_down, err_up = [], []
    for y in values:
        n = float(y)
        if n < 0:
            raise ValueError(f"Negative data bin content: {n}")
        if n == 0.0:
            low = 0.0
        else:
            low = 0.5 * ROOT.Math.chisquared_quantile(alpha / 2.0, 2.0 * n)
        up = 0.5 * ROOT.Math.chisquared_quantile_c(alpha / 2.0, 2.0 * (n + 1.0))
        err_down.append(n - low)
        err_up.append(up - n)
    return err_down, err_up


def read_hist_1d(reader, dir_path, hist_name):
    names = hist_name if isinstance(hist_name, (list, tuple)) else [hist_name]
    tried = []
    last_exc = None
    for n in names:
        path = f"{dir_path}/{n}" if dir_path else n
        tried.append(path)
        try:
            return reader.read_hist_1d(path)
        except Exception as e:
            last_exc = e

        if dir_path and "/" in dir_path:
            tail = dir_path.split("/", 1)[-1]
            path2 = f"{tail}/{n}"
            tried.append(path2)
            try:
                return reader.read_hist_1d(path2)
            except Exception as e:
                last_exc = e

    raise RuntimeError(f"Could not find histogram(s) {names}. Tried: {tried}") from last_exc


def make_data_variable(label, hist_dict):
    y = hist_dict["y"]
    v = Variable(label, is_independent=False, is_binned=False)
    v.values = y

    if USE_GARWOOD:
        dn, up = poisson_garwood_intervals(y)
        unc = Uncertainty("stat", is_symmetric=False)
        unc.values = [(-a, +b) for a, b in zip(dn, up)]
    else:
        unc = Uncertainty("stat", is_symmetric=True)
        unc.values = [math.sqrt(val) if val >= 0.0 else 0.0 for val in y]

    v.add_uncertainty(unc)
    return v


def make_mc_variable(label, hist_dict):
    y = hist_dict["y"]
    dy = hist_dict.get("dy")

    if ROUND_ROOT_TABLES and dy is not None:
        y2 = []
        dy2 = []
        if len(dy) > 0 and isinstance(dy[0], tuple):
            for yi, dyi in zip(y, dy):
                ryi, rdy = _cms_round_value_and_unc(yi, (abs(dyi[0]), abs(dyi[1])))
                y2.append(ryi)
                dy2.append(rdy)
            y, dy = y2, dy2
        else:
            for yi, dyi in zip(y, dy):
                ryi, rdy = _cms_round_value_and_unc(yi, abs(dyi))
                y2.append(ryi)
                dy2.append(rdy)
            y, dy = y2, dy2

    v = Variable(label, is_independent=False, is_binned=False)
    v.values = y

    if dy is not None:
        if len(dy) > 0 and isinstance(dy[0], tuple):
            unc = Uncertainty("stat+syst", is_symmetric=False)
            unc.values = dy
        else:
            unc = Uncertainty("stat+syst", is_symmetric=True)
            unc.values = dy
        v.add_uncertainty(unc)
    return v


def build_table_for_region(reader, cfg):
    data = read_hist_1d(reader, cfg["root_dir"], cfg["data_hist"])

    x = Variable(OBS_NAME, is_independent=True, is_binned=True, units=OBS_UNITS)
    x.values = data["x_edges"]

    table = Table(cfg["name"])
    table.location = cfg["location"]
    table.description = cfg["description"]

    table.keywords["cmenergies"] = [CM_ENERGY_GEV]
    table.keywords["reactions"] = ["P P --> Z GAMMA"]
    table.keywords["observables"] = ["N"]
    table.keywords["phrases"] = ["Event yields", "Post-fit", "Z to invisible", "High-pt photon", "Run 2"]

    table.add_variable(x)
    table.add_variable(make_data_variable("Observed", data))

    for label, hn in cfg.get("primary_mc", []):
        table.add_variable(make_mc_variable(label, read_hist_1d(reader, cfg["root_dir"], hn)))

    for label, hn in cfg.get("other_mc", []):
        table.add_variable(make_mc_variable(label, read_hist_1d(reader, cfg["root_dir"], hn)))

    for label, hn in cfg.get("extra_mc", []):
        table.add_variable(make_mc_variable(label, read_hist_1d(reader, cfg["root_dir"], hn)))

    return table


def make_asym_variable(label, values, err_down, err_up, units=None, unc_name="total"):
    v = Variable(label, is_independent=False, is_binned=False, units=units)
    v.values = values
    unc = Uncertainty(unc_name, is_symmetric=False)
    unc.values = [(-dn, +up) for dn, up in zip(err_down, err_up)]
    v.add_uncertainty(unc)
    return v


def build_table3_fiducial_xs():
    table = Table("Table3_FiducialCrossSections")
    table.location = "Table 3"
    table.description = (
        "Measured and predicted fiducial cross sections (fb) in the barrel, endcaps, and combined phase space. "
        "Predictions at NLO (MADGRAPH5_aMC@NLO) and NNLO (MATRIX)."
    )
    table.keywords["cmenergies"] = [CM_ENERGY_GEV]
    table.keywords["reactions"] = ["P P --> Z GAMMA"]
    table.keywords["observables"] = ["SIG"]
    table.keywords["phrases"] = ["Fiducial cross section"]

    reg = Variable("Region", is_independent=True, is_binned=False)
    reg.values = [
        r"Barrel ($|\eta|<1.4442$)",
        r"Endcaps ($1.566<|\eta|<2.5$)",
        "Total",
    ]
    table.add_variable(reg)

    table.add_variable(make_asym_variable(
        "Measured", [16.7, 7.8, 23.3],
        err_down=[1.0, 0.7, 1.3],
        err_up=[1.0, 0.8, 1.4],
        units="fb", unc_name="total"
    ))
    table.add_variable(make_asym_variable(
        r"NLO (MADGRAPH5_aMC@NLO)", [19.6, 6.4, 26.1],
        err_down=[0.7, 0.3, 1.0],
        err_up=[0.7, 0.3, 1.0],
        units="fb", unc_name="theory"
    ))
    table.add_variable(make_asym_variable(
        r"NNLO (MATRIX)", [19.3, 6.21, 25.4],
        err_down=[0.3, 0.09, 0.3],
        err_up=[0.3, 0.07, 0.4],
        units="fb", unc_name="theory"
    ))
    return table


def build_table4_xs_vs_ptgamma():
    table = Table("Table4_CrossSectionVsPtGamma")
    table.location = "Table 4"
    table.description = "Measured and predicted cross sections (fb) in bins of photon transverse momentum."
    table.keywords["cmenergies"] = [CM_ENERGY_GEV]
    table.keywords["reactions"] = ["P P --> Z GAMMA"]
    table.keywords["observables"] = ["SIG"]
    table.keywords["phrases"] = ["Differential cross section"]

    x = Variable(r"$p_{T}^{\gamma}$", is_independent=True, is_binned=True, units="GeV")
    bins = [(225, 275), (275, 350), (350, 450), (450, 600), (600, 800), (800, 1500)]
    x.values = bins
    table.add_variable(x)

    table.add_variable(make_asym_variable(
        "Measured",
        [12.45, 6.95, 2.61, 1.08, 0.282, 0.092],
        err_down=[0.87, 0.57, 0.33, 0.20, 0.009, 0.005],
        err_up=[0.91, 0.60, 0.35, 0.21, 0.010, 0.005],
        units="fb", unc_name="total"
    ))
    table.add_variable(make_asym_variable(
        r"NLO (MADGRAPH5_aMC@NLO)",
        [12.88, 8.14, 3.34, 1.26, 0.345, 0.107],
        err_down=[0.49, 0.30, 0.11, 0.05, 0.019, 0.012],
        err_up=[0.42, 0.31, 0.17, 0.08, 0.026, 0.011],
        units="fb", unc_name="theory"
    ))
    table.add_variable(make_asym_variable(
        r"NNLO (MATRIX)",
        [12.83, 7.89, 3.22, 1.22, 0.331, 0.091],
        err_down=[0.15, 0.15, 0.06, 0.02, 0.005, 0.014],
        err_up=[0.17, 0.16, 0.07, 0.02, 0.007, 0.020],
        units="fb", unc_name="theory"
    ))
    return table


def build_table5_aNTGC_limits():
    table = Table("Table5_aNTGC_95CL")
    table.location = "Table 5"
    table.description = "Expected and observed 95% CL intervals for anomalous coupling parameters (others fixed to zero)."
    table.keywords["cmenergies"] = [CM_ENERGY_GEV]
    table.keywords["phrases"] = ["Limits", "Confidence intervals", "aNTGC"]

    p = Variable("Parameter", is_independent=True, is_binned=False)
    p.values = [
        r"$h_{3}^{\gamma}\times 10^{4}$",
        r"$h_{4}^{\gamma}\times 10^{7}$",
        r"$h_{3}^{Z}\times 10^{4}$",
        r"$h_{4}^{Z}\times 10^{7}$",
    ]
    table.add_variable(p)

    table.keywords["reactions"] = ["P P --> Z GAMMA"]
    # Optional: use only if you want this searchable as a confidence-interval observable
    table.keywords["observables"] = ["CL"]

    exp_lo = Variable("Expected lower", is_independent=False, is_binned=False)
    exp_hi = Variable("Expected upper", is_independent=False, is_binned=False)
    obs_lo = Variable("Observed lower", is_independent=False, is_binned=False)
    obs_hi = Variable("Observed upper", is_independent=False, is_binned=False)

    exp_lo.values = [-2.8, -5.9, -1.8, -3.7]
    exp_hi.values = [2.9, 6.0, 1.9, 3.7]
    obs_lo.values = [-3.4, -6.8, -2.2, -4.1]
    obs_hi.values = [3.5, 6.8, 2.2, 4.2]

    table.add_variable(exp_lo)
    table.add_variable(exp_hi)
    table.add_variable(obs_lo)
    table.add_variable(obs_hi)
    return table


def build_submission():
    sub = Submission()
    sub.comment = (
        f"{PAPER_TITLE}. {CMS_ANALYSIS_ID}. "
        "This submission contains post-fit reconstruction-level $p_{T}^{\\gamma}$ spectra (Figure 4), "
        "fiducial and binned cross sections (Tables 3–4), and aNTGC 95% CL intervals (Table 5)."
    )
    if ABSTRACT_FILE and os.path.exists(ABSTRACT_FILE):
        sub.read_abstract(ABSTRACT_FILE)

    reader = RootFileReader(ROOT_FILENAME)
    for cfg in REGIONS:
        sub.add_table(build_table_for_region(reader, cfg))

    sub.add_table(build_table3_fiducial_xs())
    sub.add_table(build_table4_xs_vs_ptgamma())
    sub.add_table(build_table5_aNTGC_limits())
    return sub


def validate_yaml(output_dir=OUTPUT_DIR):
    try:
        import yaml
    except ImportError:
        return
    for path in sorted(glob.glob(os.path.join(output_dir, "*.yaml"))):
        with open(path, "r") as f:
            docs = list(yaml.safe_load_all(f))
        if not docs:
            raise RuntimeError(f"Empty YAML: {path}")
        if os.path.basename(path) == "submission.yaml":
            continue
        doc = docs[0]
        indep = doc.get("independent_variables", [])
        dep = doc.get("dependent_variables", [])
        if not indep or not dep:
            continue
        n = len(indep[0].get("values", []))
        for dv in dep:
            m = len(dv.get("values", []))
            if m != n:
                raise RuntimeError(f"Bin/value mismatch in {path}: {m} vs {n}")


def main():
    sub = build_submission()
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    sub.create_files(OUTPUT_DIR)
    validate_yaml(OUTPUT_DIR)
    print(f"Wrote HEPData YAML to: {OUTPUT_DIR}")


if __name__ == "__main__":
    main()


Wrote HEPData YAML to: hepdata_output
